In [3]:

!pip install pandas
!pip install numpy
! pip install matplotlib
! pip install sklearn
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# importing the necessary libraries
import os 
import io
import pandas as pd
import numpy as np
import math 
import datetime as dt
import matplotlib.pyplot as plt

In [5]:
# for evaluation we will use these libraries
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score,r2_score
from sklearn.metrics import mean_poisson_deviance,mean_gamma_deviance,accuracy_score
from sklearn.preprocessing import MinMaxScaler 

In [6]:
# for model building we will use these libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM


In [7]:
# for plotting we will use these library
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import  plotly.express as px
from plotly.subplots import make_subplots

In [8]:
# loading dataset

In [11]:
# load our dataset
# i have downloaded my dataset from yahoo finance of bitcoin in a csv file
maindf = pd.read_csv('/content/BTC-USD.csv')
print('Total number of days present in the dataset: ',maindf.shape[0])
print('Total number of fields present in the dataset: ',maindf.shape[1])

maindf.shape


Total number of days present in the dataset:  2977
Total number of fields present in the dataset:  7


(2977, 7)

In [12]:
maindf.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [13]:
maindf.tail()


,Date,Open,High,Low,Close,Adj Close,Volume
2972,2022-11-06,21285.056641,21345.376953,20920.191406,20926.486328,20926.486328,35082693210
2973,2022-11-07,20924.621094,21053.246094,20489.972656,20602.816406,20602.816406,53510852236
2974,2022-11-08,20600.671875,20664.607422,17603.544922,18541.271484,18541.271484,118992465607
2975,2022-11-09,18543.761719,18590.458984,15682.692383,15880.780273,15880.780273,102905151606
2976,2022-11-10,15877.056641,17861.927734,15877.056641,17453.900391,17453.900391,91791572992


In [14]:
maindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2977 entries, 0 to 2976
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2977 non-null   object 
 1   Open       2977 non-null   float64
 2   High       2977 non-null   float64
 3   Low        2977 non-null   float64
 4   Close      2977 non-null   float64
 5   Adj Close  2977 non-null   float64
 6   Volume     2977 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 162.9+ KB


In [15]:

maindf.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2977.000000,2977.000000,2977.000000,2977.000000,2977.000000,2.977000e+03
mean,12790.550384,13117.313868,12424.927470,12794.727548,12794.727548,1.624657e+10
std,16320.013264,16738.646239,15835.315061,16315.097104,16315.097104,2.010535e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,661.992981,672.929016,653.395020,663.255005,663.255005,9.642610e+07
50%,6867.440430,7109.560059,6709.919922,6880.323242,6880.323242,6.900880e+09
75%,15877.056641,16463.177734,15202.799805,15880.780273,15880.780273,2.803741e+10
max,67549.734375,68789.625000,66382.062500,67566.828125,67566.828125,3.509679e+11


In [16]:
# checking for null values
print('Null Values:',maindf.isnull().values.sum())
print('NA values:',maindf.isnull().values.any())
maindf.shape

Null Values: 0
NA values: False


(2977, 7)

In [17]:
# Exploratory data analysis
# print the start date and end date of the datset
sd = maindf.iloc[0][0]
ed = maindf.iloc[-1][0]

print('Starting date',sd)
print('Ending date',ed)

Starting date 2014-09-17
Ending date 2022-11-10


In [18]:
# Stock price analysis from start
# analysis of year 2014
maindf['Date'] = pd.to_datetime(maindf['Date'],format = '%Y-%m-%d')
y_2014 = maindf.loc[(maindf['Date']>='2014-09-17') & (maindf['Date'] < '2014-12-31')]
y_2014.drop(y_2014[['Adj Close','Volume']],axis=1) 

,Date,Open,High,Low,Close
0,2014-09-17,465.864014,468.174011,452.421997,457.334015
1,2014-09-18,456.859985,456.859985,413.104004,424.440002
2,2014-09-19,424.102997,427.834991,384.532013,394.795990
3,2014-09-20,394.673004,423.295990,389.882996,408.903992
4,2014-09-21,408.084991,412.425995,393.181000,398.821014
...,...,...,...,...,...
100,2014-12-26,319.152008,331.424011,316.627014,327.924011
101,2014-12-27,327.583008,328.911011,312.630005,315.863007
102,2014-12-28,316.160004,320.028015,311.078003,317.239014
103,2014-12-29,317.700989,320.266998,312.307007,312.670013


In [19]:
monthvise= y_2014.groupby(y_2014['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
# by using reindex() method and by specifying the axis we want to reindex. Default values in the new index that are not present in the dataframe are assigned NaN
monthvise = monthvise.reindex(new_order, axis=0)
monthvise


,Open,Close
Date,,
January,NaN,NaN
February,NaN,NaN
March,NaN,NaN
April,NaN,NaN
May,NaN,NaN
June,NaN,NaN
July,NaN,NaN
August,NaN,NaN
September,412.654003,407.182428
